# ABBA cell count analysis

This notebook is the last step in the ABBA whole-brain cell counting analysis.  
It assumes you have done the following steps:
- Alignment of brain slices in ABBA, exported to a QuPath project.
- Detected cells of interest in QuPath. The detections should be exported to ```.csv``` files (one per slice) in a folder called ```results```. 
- If there are regions to exclude, you should have drawn them and exported to ```.txt``` files (one per slice) in a folder called ```regions_to_exclude```.

Run this notebook to load the cell counts and do analysis on them. 

## Before we start ...
The majority of the functions and classes we need written in 3 files: ```brain_hierarchy.py```, ```readCSV_helpers.py``` and ```pls_helpers.py```. We will now import the necessary functions and classes from these python files to this notebook, so that we can use them later:

In [ ]:
import braian

And we'll need other python functions to easily read and manipulate data and make nice plots:

In [ ]:
import pandas as pd
import os

## The Allen Brain Atlas

We start by importing the mouse Allen Brain Atlas, in which we find information about all brain regions (their parent region and children regions in the brain hierarchy, for example).

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = "./data/AllenMouseBrainOntology.json"

branches_to_exclude = ['retina','VS','grv','fiber tracts']
AllenBrain = braian.AllenBrainHierarchy(path_to_allen_json, branches_to_exclude)

#edges = AllenBrain.edges_dict
#tree = AllenBrain.tree_dict
#brain_region_dict = AllenBrain.brain_region_dict
#regions = list(brain_region_dict.keys())

We can also visualize the hierarchy of brain regions as a network (a tree). **Note that running the above cell may take a few minutes**.

In [ ]:
## Plot brain region hierarchy
## If you want to plot it, install PyDot (pydot)
#fig = AllenBrain.plot_plotly_graph()
#fig.show()

Based on the graph above, you might want to specify the regions on which you want to do further PLS analysis:  
*Note: to see more information about the regions, hover over them with your mouse.*

- Specify a level. Analysis can only be done on one level (slice) in the brain region.

- To exclude brain regions that belong to a certain branch, add the *abbreviated* nodes at the beginning of the branches to the list above.  
Example:  
```branches_to_exclude = ['retina', 'VS']```  
means that **all the subregions that belong to the retina and the ventricular systems** are excluded from the PLS analysis.

## Load data

Now, we're ready to read the ```.csv``` files with the cell counts, and also the exclusion files (if there were regions to exclude).  
Below, you have to specify:
- ```animals_root```: Absolute path to the folder that contains the animal folders.
- ```group_1_dirs```: A list of names of the folders corresponding to animals in **Group 1** (e.g., Control group). Indeed, it is necessary to store the results in individual folders for each animal.
- ```group_2_dirs```: A list of names of the folders corresponding to animals in **Group 2** (e.g., Stress group).
- ```group_1_name```: A meaningful string for Group 1.
- ```group_2_name```: A meaningful string for Group 2.
- ```area_key```: A string of the column in the ```.csv``` files that refers to the size of a brain areatra
- ```tracer_key```: A string of the column in the ```.csv``` files that refers to the tracer number used to highlight the marker
- ```marker_key```: A string of the marker we would like to highlight (e.g. CFos)

Provare a modificar per ottenere densita in mm^2 (da micron)

In [ ]:
# ####################################### SET PARAMETERS ####################################


animals_root = './data/QuPath_output/'
group_1_dirs = ['Control_17C', 'Control_18C', 'Control_19C']
group_1_name = 'Control'
group_2_dirs = ['Stress_5S', 'Stress_8S', 'Stress_10S', 'Stress_13S', 'Resilient_1R', 'Resilient_2R', 'Resilient_3R', 'Resilient_4R', 'Resilient_11R']
group_2_name = 'Stress'
area_key = 'Area um^2'
# area_key = 'DAPI: DAPI area um^2'
tracer_key='Num AF647'
marker_key='CFos'

data_output_path = './data/python_norm_output/'
plots_output_path = './plots/python_output/'


# ###########################################################################################


if not(os.path.exists(data_output_path)):
    os.makedirs(data_output_path, exist_ok=True)
if not(os.path.exists(plots_output_path)):
    os.makedirs(plots_output_path, exist_ok=True)

In [ ]:
old_df = pd.read_csv("./data/python_norm_output_old/Control_17C/Control_17C_sum.csv", sep="\t", index_col="Class")
old = set(old_df.index)
len(old)

In [ ]:
slice = braian.BrainSlice(AllenBrain, "./data/QuPath_output/Control_17C/results/2022_04_22__12_28__2554.czi - Scene #06_regions.txt",
                           "./data/QuPath_output/Control_17C/regions_to_exclude/2022_04_22__12_28__2554.czi - Scene #06_regions_to_exclude.txt",
                           "Control_17C", area_key, tracer_key, marker_key, area_units="µm2")
slice.data

In [ ]:
sliced_brain = braian.SlicedBrain(group_1_dirs[0],
                                        os.path.join(animals_root, group_1_dirs[0]),
                                        AllenBrain,
                                        area_key,
                                        tracer_key,
                                        marker_key,
                                        area_units="µm2")
df = pd.concat([slice.data for slice in sliced_brain.slices])
df = df.groupby(df.index, axis=0).sum()
new = set(df.index)
len(new)

In [ ]:
sliced_brain.slices[0].data
[len(slice.data) for slice in sliced_brain.slices]

Now, we load the Control and Stress results seperately in two pandas dataframes, and save the results.

**Note**: regions to exclude are automatically excluded.

In [ ]:
group_1_slices = [braian.SlicedBrain(animal_dir,
                                        os.path.join(animals_root, animal_dir),
                                        AllenBrain,
                                        area_key,
                                        tracer_key,
                                        marker_key,
                                        area_units="µm2")
                    for animal_dir in group_1_dirs]
print(f'Imported all brain slices from {str(len(group_1_slices))} animals of {group_1_name} group.')

group_2_slices = [braian.SlicedBrain(animal_dir,
                                        os.path.join(animals_root, animal_dir),
                                        AllenBrain,
                                        area_key,
                                        tracer_key,
                                        marker_key,
                                        area_units="µm2")
                    for animal_dir in group_2_dirs]
print(f'Imported all brain slices  {str(len(group_2_slices))} animals of {group_2_name} group.')

In [ ]:
cvar_brains = [braian.AnimalBrain(sliced_brain, marker_key, mode="cvar", hemisphere_distinction=False) for sliced_brain in group_1_slices+group_2_slices]
AllenBrain.select_from_csv("./data/AllenSummaryStructures.csv")
cvar_brains = [braian.filter_selected_regions(brain, AllenBrain) for brain in cvar_brains]

cv_threshold = 1
print("N regions above threshold:", sum([(brain.data > cv_threshold).sum() for brain in cvar_brains]))
print("N regions below threshold:", sum([(brain.data <= cv_threshold).sum() for brain in cvar_brains]))
braian.plot_cv_above_threshold([brain.data for brain in cvar_brains], group_1_dirs+group_2_dirs, marker_key, cv_threshold=cv_threshold).show()

In [ ]:
#r = 'IG'
#group = group_2_slices
#dirs = group_2_dirs
#i_animal = 7
#df = pd.concat([braian.merge_hemispheres(slice) for slice in group[i_animal]])
#slices_per_area = df.groupby(df.index).count().iloc[:,0]
#print(f"""Summary for brain region '{r}' of {dirs[i_animal]}:
#    - N slices: {slices_per_area[r]}
#    - Mean: {braian.reduce_brain_densities(group[i_animal], marker_key, "avg")[r]:.2f} {marker_key}/mm²),
#    - S.D.: {braian.reduce_brain_densities(group[i_animal], marker_key, "std")[r]:.2f} {marker_key}/mm²,
#    - Coefficient of Variation: {braian.reduce_brain_densities(group[i_animal], marker_key, "cvar")[r]}
#""")

In [ ]:
# NOTE: brains are being written WITH Left/Right discrimination
# If you desire to save them without, call AnimalBrain with hemisphere_distinction=False
group_1_sum_brains = [braian.AnimalBrain(sliced_brain, marker_key) for sliced_brain in group_1_slices]
for animal in group_1_sum_brains:
    animal.write_brains(data_output_path)

group_2_sum_brains = [braian.AnimalBrain(sliced_brain, marker_key) for sliced_brain in group_2_slices]
for animal in group_2_sum_brains:
    animal.write_brains(data_output_path)

In [ ]:
group_1 = braian.AnimalGroup(group_1_name, group_1_sum_brains, marker_key, AllenBrain)
group_2 = braian.AnimalGroup(group_2_name, group_2_sum_brains, marker_key, AllenBrain)

In [ ]:
# Save results
group_1.save(data_output_path, f"results_cell_counts_{group_1.name}.csv")
group_2.save(data_output_path, f"results_cell_counts_{group_2.name}.csv")